# 가우시안 필터링
- 스무딩(smoothing)은 영상의 세세한 부분을 회선을 통해서 부드럽게 하는 기법으로 블러링과 같은 의미이다. 스무딩 처리에 사용되는 대표적인 방법으로 가우시안 필터링이 있다 
- 가우시안 필터링은 가우시안 분포를 마스크의 게수로 사용하여 회선을 수행하는 것을 말한다 
- 가우시안 분포는 같은 표현으로 정규 분포(normal distribution)이라고 한다 
- 정규 분포는 특정 값의 출현 비율을 그래프로 그렸을 때, 평균에서 가장 큰 수치를 가지며, 평균의 기준으로 좌우 대칭 형태가 나타나고, 좌우 양끝으로 갈수록 급격하게 수치가 낮아지는 종의 모양의 형태를 보인다 

- 정보분포를 평균과 표준 편차를 이용해서 나타낼수 있으며, 표준편차가 커지면 그래프의 폭이 널어지고 표준편차가 작아지면 폭이 좁아진다.
![noraml](img/normal.png)

- 가우시안 분포를 회선 마스크로 적용하려면 2차원 행렬로 구성해야 한다 
![gaussian](img/2gaussian.jpg)


- OpenCV에서 cv2.GaussianBlur() 함수로 가우시안 스무딩을 수행한다 
- 다른 방법으로 cv2.sepFilter2D() 함수에 1차원 가우시안 계수를 적용 가능하다. 이 때 1차원 가우시안 계수는 cv2.getGaussianKernel() 함수로 생성한다 

In [1]:
import numpy as np, cv2

In [2]:
def getGaussianMask(ksize, sigmaX, sigmaY):
    sigma = 0.3 * ((np.array(ksize) - 1.0) * 0.5 - 1.0) + 0.8  # 표준 편차
    if sigmaX <= 0: sigmaX = sigma[0]
    if sigmaY <= 0: sigmaY = sigma[1]

    u = np.array(ksize)//2            # 커널 크기 절반
    x = np.arange(-u[0], u[0]+1, 1)   # x 방향 범위
    y = np.arange(-u[1], u[1]+1, 1)   # y 방향 범위
    x, y = np.meshgrid(x, y)          # 정방 행렬 생성

    ratio = 1 / (sigmaX*sigmaY * 2 * np.pi)
    v1 = x ** 2 / (2 * sigmaX ** 2)
    v2 = y ** 2 / (2 * sigmaY ** 2 )
    mask = ratio * np.exp(-(v1+v2))  # 2차원 정규 분포 수식 
    return mask / np.sum(mask)

In [3]:
image = cv2.imread("img/smoothing.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상파일 읽기 오류")

In [4]:
ksize = (5, 17)                                        # 크기는 가로x세로로 표현
gaussian_2d = getGaussianMask(ksize, 0, 0)
gaussian_1dX = cv2.getGaussianKernel(ksize[0], 0, cv2.CV_32F)   # 가로 방향 마스크
gaussian_1dY = cv2.getGaussianKernel(ksize[1], 0, cv2.CV_32F)   # 세로 방향 마스크

gauss_img1 = cv2.filter2D(image, -1, gaussian_2d)     # 사용자 생성 마스크 적용
gauss_img2 = cv2.GaussianBlur(image, ksize, 0)
gauss_img3 = cv2.sepFilter2D(image, -1, gaussian_1dX, gaussian_1dY)

In [5]:
titles = ['image','gauss_img1','gauss_img2','gauss_img3']
[cv2.imshow(t, eval(t)) for t in titles]
cv2.waitKey(0)

-1

![result](img/12_result.png)